<a id='GoTop'></a>
# Digging Through Source of Hadrontherapy Geant4 Simulation

* Main File ['hadrontherapy.cc'](#hadrontherapy.cc) 
* Header files in include/ sub-directory
    * [HadrontherapyGeometryController.hh](#HadrontherapyGeometryController.hh)
* Source files in src/ sub-directory
    * [HadrontherapyGeometryController.cc](#HadrontherapyGeometryController.cc)

In [1]:
!pwd

/Users/kpadhikari/KpInstalls/Geant4/ExamplesBld/HadronTherapyNew/Source/hadrontherapy


In object oriented program is a program consisting of interactive objects.

<a id='GoTop'></a><a id="hadrontherapy.cc"></a>
## Main File hadrontherapy.cc
* **Object and pointer creation of**
    * **G4UIExecutive**
        - To detect interactive mode (if no arguments) and define UI session
        - If no argument or .mac file is given, a G4UIExecutive object and its **pointer 'ui'** is created as **ui = new G4UIExecutive(argc, argv);**
    * **G4Timer** 
        - then Start() method is invoked.
    * **Random engine is set/initialized** (also for multithreading) using **CLHEP::RanluxEngine** and **G4Random**. 
        - **time(NULL)** is used for seed.
    * either **G4MTRunManager** (for multi-threading) or **G4RunManager**.
        * Please note that runManager is a singleton type see the class description in [G4RunManager.hh](https://gitlab.cern.ch/geant4/geant4/blob/bc13104726e8975c0e4a62bf0d859dcca77d4213/source/run/include/G4RunManager.hh) (or [this](http://www.apc.univ-paris7.fr/~franco/g4doxy4.10/html/_g4_run_manager_8hh_source.html)) and see [this code](#HadrontherapyGeometryController::registerGeometry) where we have a local pointer which is accessing the same object (first created here in the main()). The class description says the following:
            - G4RunManager is the only manager class in Geant4 kernel which the user MUST construct an object by him/herself in the main(). Also, G4RunManager is the only manager class in Geant4 kernel which the user CAN derive it to costomize the behavior of the run control. For this case, user should use protected methods provided in this class for procedures he/she does not want to change.
            - G4RunManager or the derived class of it MUST be a singleton. The user MUST NOT construct more than one object even if there are two different concrete implementations.
        * The method G4RunManager::GetRunManager() ([see G4RunManager.cc](http://www.apc.univ-paris7.fr/~franco/g4doxy4.10/html/_g4_run_manager_8cc_source.html#l00074)), just returns **fRunManager**, which is assigned 'this' insided the constructor G4RunManager::G4RunManager(). Now, if we go back to G4RunManager.hh, we see the **private static pointer** declared as follows: **static G4ThreadLocal G4RunManager* fRunManager;**, which ensures that we have that pointer residing in global scope (because it's static) and thus accessible from anywhere. 
    * [**HadrontherapyGeometryController**](#HadrontherapyGeometryController.hh) and [**HadrontherapyGeometryMessenger**](#HadrontherapyGeometryMessenger.hh) (takes the pointer of the former as input).
    * **G4ScoringManager** (via GetScoringManager() because it is a **singleton object**, where constructor is private).
* **Initialize**
    * **Geometry:** geometryController->SetGeometry("default");
        - Inside [SetGeometry](#HadrontherapyGeometryController.cc) method, it calls **registerGeometry(new PassiveProtonBeamLine());** for 'default' option. Other options are 'Carbon', 'LaserDriven', 'TrentoLine'.
    * **the physics:**  runManager->SetUserInitialization(phys);
        - **(G4VModularPhysicsList* phys)** gets value either via PHYSLIST env variable (using G4PhysListFactory) or via an object of HadrontherapyPhysicsList. 
    * **the Actions:**  runManager->SetUserInitialization(new HadrontherapyActionInitialization);
    * **Command based scoring:** G4ScoringManager::GetScoringManager();
    * **Interaction data: stopping powers:**
        - HadrontherapyInteractionParameters* pInteraction = new HadrontherapyInteractionParameters(true);
    * **Initialize analysis:** 
        - HadrontherapyAnalysis* analysis = HadrontherapyAnalysis::GetInstance();
    * **Visualization:**
        - G4VisManager* visManager = new G4VisExecutive;
        - visManager -> Initialize();
* **Create and Use User Interface manager**
    * Get the pointer to the User Interface manager (kp: again singleton)
        * G4UImanager* UImanager = G4UImanager::GetUIpointer();
    * if (!ui) // **batch mode**
        * Pass **"/control/execute argv[1]"** command to UImanager:
            - UImanager->ApplyCommand("/control/execute" + fileName); 
    * else     // **interactive mode**
        * UImanager->ApplyCommand("/control/execute init_vis.mac");
        * ui->SessionStart();
        * delete ui;
* **Wrap up/Close out and Delete things**
    * delete visManager;
    * theTimer->Stop();
    * if ( HadrontherapyMatrix * pMatrix = HadrontherapyMatrix::GetInstance() ) pMatrix -> StoreDoseFluenceAscii();
    * if (HadrontherapyLet *let = HadrontherapyLet::GetInstance())
        if(let -> doCalculation)
        {
            let -> LetOutput(); 	// Calculate let
            let -> StoreLetAscii(); // Store it
        }
    * delete geometryMessenger;
    * delete geometryController;
    * delete pInteraction;
    * delete runManager;
    * delete analysis;
    * return 0;

[Go Top](#GoTop)<a id='HadrontherapyGeometryController.hh'></a>
## include/HadrontherapyGeometryController.hh
```cpp
// Hadrontherapy advanced example for Geant4
// See more at: https://twiki.cern.ch/twiki/bin/view/Geant4/AdvancedExamplesHadrontherapy
#ifndef HadrontherapyGeometryController_hh
#define HadrontherapyGeometryController_hh 1

#include "globals.hh"
#include "G4String.hh"
#include "G4VUserDetectorConstruction.hh"

/**
 * Controller for geometry selection
 *
 * This controller is called by the geometry messenger and used to
 * select the geometry. Each available geometry must have unique name
 * and it must be known by the geometry controller.
 */
class HadrontherapyGeometryController
{
public:
  HadrontherapyGeometryController();
  ~HadrontherapyGeometryController();

  /**
   * Select a geometry by name.
   */
  void SetGeometry(G4String);

private:
  void registerGeometry(G4VUserDetectorConstruction *detector);
};

#endif
```

[Go Top](#GoTop)<a id='HadrontherapyGeometryController.cc'></a>
## src/HadrontherapyGeometryController.cc
```cpp
// Hadrontherapy advanced example for Geant4
// See more at: https://twiki.cern.ch/twiki/bin/view/Geant4/AdvancedExamplesHadrontherapy

#include "HadrontherapyGeometryController.hh"
#include "HadrontherapyDetectorConstruction.hh"
#include "HadrontherapyInteractionParameters.hh"
#include "HadrontherapyDetectorROGeometry.hh"
#include "HadrontherapyTIFPAPassiveProtonBeamLine.hh"
#include "PassiveProtonBeamLine.hh"
#include "PassiveCarbonBeamLine.hh"
#include "LaserDrivenBeamLine.hh"
#include "G4RunManager.hh"
#include "G4VUserParallelWorld.hh"
#include "G4ThreeVector.hh"

/////////////////////////////////////////////////////////////////////////////
HadrontherapyGeometryController::HadrontherapyGeometryController()
{}

/////////////////////////////////////////////////////////////////////////////
HadrontherapyGeometryController::~HadrontherapyGeometryController()
{}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyGeometryController::SetGeometry(G4String name)
{
    G4cout <<"Activating geometry " << name << G4endl;
    if(name == "default")
    {
      registerGeometry(new PassiveProtonBeamLine());
    }
    else if(name == "Carbon")
    {
      registerGeometry(new PassiveCarbonBeamLine());
    }
    else if(name == "LaserDriven")
    {
      registerGeometry(new LaserDrivenBeamLine());
    }
    
    else if(name == "TrentoLine")
    {
        registerGeometry(new TrentoPassiveProtonBeamLine());
    }
    else
    {
        G4cout <<"Unknown geometry: " << name << ". Geometry not changed." << G4endl;
    }
}
```
<a id="HadrontherapyGeometryController::registerGeometry"></a>
```cpp
/////////////////////////////////////////////////////////////////////////////
void HadrontherapyGeometryController::registerGeometry(G4VUserDetectorConstruction *detector)
{
	G4RunManager *runManager = G4RunManager::GetRunManager();
	runManager -> SetUserInitialization(detector);
	runManager -> GeometryHasBeenModified();
}
```

[Go Top](#GoTop)<a id='HadrontherapyGeometryMessenger.hh'></a>
## include/HadrontherapyGeometryMessenger.hh

```cpp
#ifndef HadrontherapyGeometryMessenger_h
#define HadrontherapyGeometryMessenger_h 1

#include "globals.hh"
#include "G4UImessenger.hh"
#include "G4String.hh"

class HadrontherapyGeometryController;
class G4UIdirectory;
class G4UIcmdWithADoubleAndUnit;
class G4UIcmdWithAString;

class HadrontherapyGeometryMessenger: public G4UImessenger
{
public:
  HadrontherapyGeometryMessenger(HadrontherapyGeometryController* );
  ~HadrontherapyGeometryMessenger();
    
  void SetNewValue(G4UIcommand*, G4String);
    
private:

  // Pointer to the detector component
  HadrontherapyGeometryController* hadrontherapyGeometryController;

  G4UIdirectory *changeTheGeometryDir;      ///> UI directory for the geometry control
  G4UIcmdWithAString *changeTheGeometryCmd;//, *changeTheDetectorCmd; ///> Select the geometry and the detector
};
#endif
```

[Go Top](#GoTop)<a id='HadrontherapyGeometryMessenger.cc'></a>
## src/HadrontherapyGeometryMessenger.cc
```cpp
#include "HadrontherapyGeometryMessenger.hh"
#include "HadrontherapyGeometryController.hh"
#include "G4UIdirectory.hh"
#include "G4UIcmdWithADoubleAndUnit.hh"
#include "G4UIcmdWithAString.hh"

/////////////////////////////////////////////////////////////////////////////
HadrontherapyGeometryMessenger::HadrontherapyGeometryMessenger(HadrontherapyGeometryController* controller)
:hadrontherapyGeometryController(controller)

{
    changeTheGeometryDir = new G4UIdirectory("/geometrySetup/");
    changeTheGeometryDir -> SetGuidance("Geometry setup");
    
    changeTheGeometryCmd = new G4UIcmdWithAString("/geometrySetup/selectGeometry",this);
    changeTheGeometryCmd -> SetGuidance("Select the geometry you wish to use");
    changeTheGeometryCmd -> SetParameterName("Geometry",false);
    changeTheGeometryCmd -> AvailableForStates(G4State_PreInit);
    
    /*  changeTheDetectorCmd = new G4UIcmdWithAString("/geometrySetup/selectDetector",this);
    changeTheDetectorCmd -> SetGuidance("Select the detector you wish to use");
    changeTheDetectorCmd -> SetParameterName("Detector",false);
    changeTheDetectorCmd -> AvailableForStates(G4State_PreInit);*/
}

/////////////////////////////////////////////////////////////////////////////
HadrontherapyGeometryMessenger::~HadrontherapyGeometryMessenger()
{
    delete changeTheGeometryDir;
    delete changeTheGeometryCmd;
}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyGeometryMessenger::SetNewValue(G4UIcommand* command,G4String newValue)
{

 if( command == changeTheGeometryCmd )
    {
        hadrontherapyGeometryController -> SetGeometry (newValue);
    }
}
```